<a href="https://colab.research.google.com/github/HeTalksInMaths/QOSF/blob/master/QOSF_Task_2_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We make use of the pennylane framework for our task to learn parametrs in our quantum circuit to produce 01 and 10 with 1/2 probability each.

In [1]:
#!pip install pennylane   # uncomment first hash for installation on Colab. Restart runtime after installation.
import pennylane as qml
from pennylane import numpy as np

We set the analytic parameter in our device initialization to False to introduce stochastisity in our measurements and later make use of dev.shots to adjust the number of measurements.


In [83]:
dev = qml.device("default.qubit", wires=2, analytic = False)

Let us start with a simple structure where we have expected results so that the focus is on developing intutition in the parameter learning. With two RY gates (with both qubits inititalized as 0) and a subsequent CNOT between the wires one expected result would be for the parameters to equal [pi/2, pi]. 


RY(pi/2) sends the first qubit from 0 to + (as is usually prescribed for creating a Bell State before application of the CNOT gate) and RY(pi) send the second qubit from 0 to 1. The subsequent CNOT then results in mismatched qubit values. Here this is no relative phase but different parameters may result in relative phase even though the same theoretical probability distributed will be generated.  

In [84]:
@qml.qnode(dev)
def circuit(params):
  qml.RY(params[0], wires=0)
  qml.RY(params[1], wires=1)
  qml.CNOT(wires=[0,1])
  return qml.probs(wires=[0,1])

We set the seed for reprodcubility and test our prior intution regarding the parameters with a large sample size (to get close to the theoeretical limit).

We also draw the circuit for clarity.

In [85]:
np.random.seed(0)
dev.shots = 10**6
circuit([np.pi/2, np.pi])

array([0.      , 0.499195, 0.500805, 0.      ])

In [86]:
print(circuit.draw())

 0: ──RY(1.571)──╭C──╭┤ Probs 
 1: ──RY(3.142)──╰X──╰┤ Probs 



Our intuition is correct and with the parameters [pi, pi/2] we are very close (but not identical due to sampling and stochasticity) to 0.5 probability for the 01 and 10 states. 

Note the probabilities are given to 6 decimals due to dev.shots = 10**6. We use qml.probs instead of qml.sample to save on some data processing but show below how qm.probs is noisy and is given to accuracy given by the number of samples set by dev.shots.

In [87]:
for i in [1, 10, 100, 1000]:
  dev.shots = i
  for j in [0,2]:
    np.random.seed(j)
    print("For {} samples with seed {}, probs = {}".format(i, j, circuit([np.pi/2, np.pi])))

For 1 samples with seed 0, probs = [0. 0. 1. 0.]
For 1 samples with seed 2, probs = [0. 1. 0. 0.]
For 10 samples with seed 0, probs = [0.  0.3 0.7 0. ]
For 10 samples with seed 2, probs = [0.  0.8 0.2 0. ]
For 100 samples with seed 0, probs = [0.   0.51 0.49 0.  ]
For 100 samples with seed 2, probs = [0.   0.56 0.44 0.  ]
For 1000 samples with seed 0, probs = [0.    0.517 0.483 0.   ]
For 1000 samples with seed 2, probs = [0.    0.534 0.466 0.   ]


Next we define our cost function for learning by setting the desired probability array [0, 0.5, 0.5, 0] as our target and use squre loss.

In [88]:
def cost(x):

  return sum((np.array([0, 0.5, 0.5, 0]) - np.array(circuit(x)))**2)

Before attempting to find the parameters with only one sample per iteration we can use some intution to consider potential pitfalls. After each measurement, the cost is bounded below by 0.5. As a result we need a small stepsize or update our stepsize to decrease. We do the latter as our initialized [0,0] paramaters produce [1, 0, 0, 0] which is far from our desired result. Our optimizer is gradient descent.

In [90]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(1)

# set the number of samples
dev.shots = 1

# set the number of steps
steps = 5000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)

  if i > 100:
    opt.update_stepsize(0.001)
    
  elif i > 10:
    opt.update_stepsize(0.1)

  if (i+1) % 500 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))

print("Optimized rotation angles: {}".format(params))

Cost after step   500:  0.5000000
Current rotation angles: [-1.803   3.1605]
Cost after step  1000:  0.5000000
Current rotation angles: [-1.7235  3.1505]
Cost after step  1500:  0.5000000
Current rotation angles: [-1.6665  3.1395]
Cost after step  2000:  0.5000000
Current rotation angles: [-1.6105  3.122 ]
Cost after step  2500:  0.5000000
Current rotation angles: [-1.6425  3.1375]
Cost after step  3000:  0.5000000
Current rotation angles: [-1.6405  3.127 ]
Cost after step  3500:  0.5000000
Current rotation angles: [-1.6105  3.1345]
Cost after step  4000:  0.5000000
Current rotation angles: [-1.5845  3.1365]
Cost after step  4500:  0.5000000
Current rotation angles: [-1.5765  3.147 ]
Cost after step  5000:  0.5000000
Current rotation angles: [-1.5785  3.144 ]
Optimized rotation angles: [-1.5785  3.144 ]


Let us now see how good the resulting parameters are is if we had a large number of samples.

In [91]:
params1 = params

dev.shots = 10**6
np.random.seed(0)
circuit(params1)

array([1.00000e-06, 4.95431e-01, 5.04567e-01, 1.00000e-06])

We instantiate a second device for theoretical results with the given parameters (by not setting the analytic argument to False).

In [92]:
dev2 = qml.device("default.qubit", wires=2)

@qml.qnode(dev2)
def circuit2(params):
  qml.RY(params[0], wires=0)
  qml.RY(params[1], wires=1)
  qml.CNOT(wires=[0,1])
  return qml.probs(wires=[0,1])

In [93]:
circuit2(params1)

array([7.18833647e-07, 4.96147483e-01, 5.03851069e-01, 7.29994838e-07])

In both the noisy simulator with large samples and the theoretical circuit we achieve results very close to [0, 0.5, 0.5, 0]. Our parameters are very close to [-pi/2, pi] as expected except with relative phase.

That being said this required some careful calibration with the stepsize for the optimizer. A large stepsize would have been very noisy due to cost being bounded below by 0.5 and a small stepsize would have taken many more iterations.

Note that even though the second parameter converged to pi quickly and it may appear that fewer iterations for learning may be needed, the first parameter continues to move towards -pi/2 after being at -1.8 after 500 steps. 

We tried the same stepsize update procedure for other seeds (to make sure we weren't overfitting to the data in this seed) and produced results that were also close to [0, 0.5, 0.5, 0].

In [57]:
dev2.state

array([-0.00084784+0.j,  0.70437737+0.j, -0.70982467+0.j,  0.0008544 +0.j])

The absolute value of the coefficients are close to 1/sqrt(2) but we have 01 - 10 (relative phase pi).

Next lets try 10 samples per iteration. The loss is no longer bounded below by 0.5 so we can have smaller updates to our parameters without a very small stepsize. The lowest non-zero cost is now 0.02. We may repeatedly achieve zero cost in optimization due to the large noise for 10 samples but the parameters in these instances may still be far from what we are looking for.

In [181]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.02)

# set the number of samples
dev.shots = 10

# set the number of steps
steps = 2000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)

  if (i+1) % 200 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
 
print("Optimized rotation angles: {}".format(params))

Cost after step   200:  0.9400000
Current rotation angles: [-1.1212  0.9784]
Cost after step   400:  0.0800000
Current rotation angles: [-1.5196  2.3592]
Cost after step   600:  0.0200000
Current rotation angles: [-1.5474  2.596 ]
Cost after step   800:  0.1400000
Current rotation angles: [-1.5766  2.6972]
Cost after step  1000:  0.0600000
Current rotation angles: [-1.5376  2.7568]
Cost after step  1200:  0.0200000
Current rotation angles: [-1.544  2.808]
Cost after step  1400:  0.0600000
Current rotation angles: [-1.5914  2.8238]
Cost after step  1600:  0.0200000
Current rotation angles: [-1.5818  2.8454]
Cost after step  1800:  0.0200000
Current rotation angles: [-1.5922  2.86  ]
Cost after step  2000:  0.1800000
Current rotation angles: [-1.5472  2.8906]
Optimized rotation angles: [-1.5472  2.8906]


These results initially don't look as good as the 1 sample per iteration case but by using a single stepsize during the whole optimization we are less prone to overfit the samples in our particular seed. 

We are also using less iterations (2000 versus 5000 previously).

In [182]:
params10 = params

dev.shots = 10**6
np.random.seed(0)
circuit(params10)

array([0.008002, 0.502982, 0.481343, 0.007673])

In [183]:
circuit2(params10)

array([0.00801823, 0.50377884, 0.48055434, 0.00764859])

In [184]:
dev2.state

array([ 0.08954459+0.j,  0.70977379+0.j, -0.69322027+0.j, -0.08745621+0.j])

As expected the results are a little worse than with params1 (found using one sample per iteration). We were able to create an update stepsize procedure to get even better results than params1 but weren't replicating the results with other seeds and so were likely overfitting the samples in our seed. 

With further iterations the probabilities for the states 00 and 11 continued to converge to zero as desired but the single stepsize of 0.02 was large enough that the 01 and 10 probabilities were too noisy to see much improvement. 

We highlight this by re-running our code with a print(circuit2(params)) statement to show how the theoretical probabilities of the 00 and 11 decrease with more iterations (and will continue to do so to build intution in the 100 and 1000 sample cases). Note the learning still comes from our noisy simulator - using the theoretical probabilities is purely for our understanding as the 
cost function is highly susceptible to noise and as a result we can't infer improvements from it.

In [228]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.02)

# set the number of samples
dev.shots = 10

# set the number of steps
steps = 2000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  
  if (i+1) % 200 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    print(circuit2(params))

print("Optimized rotation angles: {}".format(params))

Cost after step   200:  0.9400000
Current rotation angles: [-1.1212  0.9784]
[0.55890324 0.15839777 0.06242692 0.22027208]
Cost after step   400:  0.0800000
Current rotation angles: [-1.5196  2.3592]
[0.07641278 0.4491742  0.4054402  0.06897282]
Cost after step   600:  0.0200000
Current rotation angles: [-1.5474  2.596 ]
[0.03714412 0.47455298 0.45285697 0.03544593]
Cost after step   800:  0.1400000
Current rotation angles: [-1.5766  2.6972]
[0.02414109 0.47295709 0.47847888 0.02442294]
Cost after step  1000:  0.0600000
Current rotation angles: [-1.5376  2.7568]
[0.01888768 0.49770744 0.4657307  0.01767418]
Cost after step  1200:  0.0200000
Current rotation angles: [-1.544  2.808]
[0.01415125 0.49924531 0.47319072 0.01341272]
Cost after step  1400:  0.0600000
Current rotation angles: [-1.5914  2.8238]
[0.01226023 0.47743866 0.49752507 0.01277604]
Cost after step  1600:  0.0200000
Current rotation angles: [-1.5818  2.8454]
[0.01076654 0.48373174 0.49449562 0.01100611]
Cost after step  1

Ultimately trying a smaller initial stepsize with many many more iterations should produce more desirable results. Finding params1 with an unchagned stepsize through the optimization process and comparing the need number of iterations for a desired result would also be interesting.

We hope to return to these if time permits but leave the above portions unchanged to illustrate our learning journey through this process.

Next is 100 samples per iteration where the cost function can be significantly smaller than before when non-zero to aid in learning.

In [242]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.015)

# set the number of samples
dev.shots = 100

# set the number of steps
steps = 50000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)

  if (i+1) % 2000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    print(circuit2(params))

print("Optimized rotation angles: {}".format(params))

Cost after step  2000:  0.0098000
Current rotation angles: [1.5664755 2.8313925]
[0.01198343 0.49017697 0.48595927 0.01188032]
Cost after step  4000:  0.0000000
Current rotation angles: [1.556274  2.9381745]
[0.00522941 0.5020315  0.48765939 0.0050797 ]
Cost after step  6000:  0.0002000
Current rotation angles: [1.5743025 2.983416 ]
[0.00311002 0.49513689 0.49862117 0.00313191]
Cost after step  8000:  0.0018000
Current rotation angles: [1.5737505 3.006891 ]
[0.00225795 0.49626497 0.49920575 0.00227133]
Cost after step 10000:  0.0162000
Current rotation angles: [1.5577485 3.01899  ]
[0.00190106 0.50462267 0.49162418 0.00185209]
Cost after step 12000:  0.0008000
Current rotation angles: [1.575729 3.031392]
[0.00150901 0.49602467 0.50094236 0.00152397]
Cost after step 14000:  0.0032000
Current rotation angles: [1.567626 3.043533]
[0.00120481 0.50038035 0.49721765 0.00119719]
Cost after step 16000:  0.0018000
Current rotation angles: [1.5599625 3.051444 ]
[0.00102616 0.50439065 0.49357903 

In [244]:
params100 = params

dev.shots = 10**6
np.random.seed(0)
circuit(params100)

array([3.27000e-04, 5.00552e-01, 4.98759e-01, 3.62000e-04])

In [245]:
circuit2(params100)

array([3.43547184e-04, 5.01373613e-01, 4.97941644e-01, 3.41195558e-04])

In [246]:
dev2.state

array([0.01853503+0.j, 0.70807741+0.j, 0.7056498 +0.j, 0.01847148+0.j])

The results are good and the 01 and 10 probabilities seem to stably hover around 0.5 during the end stages of optimization.

With 1000 samples per iteration we should have the least noisy probability measurements and the lowest non-zero cost to aid in learning. As a result we can use a slightly larger step size and fewer iterations and still achieve very good results. 



In [223]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.02)

# set the number of samples
dev.shots = 1000

# set the number of steps
steps = 40000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)

  if (i+1) % 2000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    print(circuit2(params))

print("Optimized rotation angles: {}".format(params))

Cost after step  2000:  0.0009900
Current rotation angles: [ 1.5697238  -2.89160592]
[0.00777941 0.49275685 0.491701   0.00776274]
Cost after step  4000:  0.0001220
Current rotation angles: [ 1.5735341 -2.9745905]
[0.0034686  0.49516252 0.49788124 0.00348764]
Cost after step  6000:  0.0007700
Current rotation angles: [ 1.57647704 -3.00687274]
[0.00225238 0.49490727 0.50056222 0.00227812]
Cost after step  8000:  0.0002600
Current rotation angles: [ 1.56565414 -3.0255521 ]
[0.00168993 0.50088115 0.49575628 0.00167264]
Cost after step 10000:  0.0004560
Current rotation angles: [ 1.57170936 -3.03826152]
[0.00133226 0.49821122 0.49912182 0.0013347 ]
Cost after step 12000:  0.0000900
Current rotation angles: [ 1.57289338 -3.0482056 ]
[0.00108707 0.49786441 0.49995689 0.00109164]
Cost after step 14000:  0.0000700
Current rotation angles: [ 1.56914996 -3.05549692]
[0.00092751 0.49989567 0.49825235 0.00092446]
Cost after step 16000:  0.0001260
Current rotation angles: [ 1.57363952 -3.06079418]


In [224]:
params1000 = params

dev.shots = 10**6
np.random.seed(0)
circuit(params1000)

array([3.00000e-04, 4.98178e-01, 5.01183e-01, 3.39000e-04])

In [225]:
circuit2(params1000)

array([3.11978916e-04, 4.98950435e-01, 5.00424686e-01, 3.12900722e-04])

In [227]:
dev2.state

array([ 0.01766292+0.j, -0.70636424+0.j, -0.70740702+0.j,  0.017689  +0.j])

The results are very good and the 01 and 10 probabilities seem to stably hover around 0.5 during the end stages of optimization.

We now come back to the 10 samples per iteration instance. We decrease the stepsize to 0.01 (from 0.02 prior) so the updates are less noisy and use a large number of iterations for convergence.

In [229]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.01)

# set the number of samples
dev.shots = 10

# set the number of steps
steps = 60000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
 
  if (i+1) % 3000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    print(circuit2(params))

print("Optimized rotation angles: {}".format(params))

Cost after step  3000:  0.0800000
Current rotation angles: [-1.5399  2.8279]
[0.01257672 0.50286899 0.47273132 0.01182298]
Cost after step  6000:  0.0200000
Current rotation angles: [-1.5968  2.9604]
[0.00398622 0.48301341 0.50880133 0.00419904]
Cost after step  9000:  0.1800000
Current rotation angles: [-1.5723  3.025 ]
[0.00169475 0.49755341 0.49905198 0.00169986]
Cost after step 12000:  0.0200000
Current rotation angles: [-1.5233  3.0397]
[0.0013582  0.52238103 0.47502569 0.00123508]
Cost after step 15000:  0.0200000
Current rotation angles: [-1.5621  3.0127]
[0.00209183 0.50225628 0.49359613 0.00205576]
Cost after step 18000:  0.0200000
Current rotation angles: [-1.5331  3.0397]
[0.00134551 0.51749819 0.47990853 0.00124777]
Cost after step 21000:  0.0200000
Current rotation angles: [-1.5605  3.0117]
[0.00212773 0.50302034 0.49276756 0.00208436]
Cost after step 24000:  0.0000000
Current rotation angles: [-1.5955  3.0484]
[0.00105803 0.48659139 0.51123896 0.00111162]
Cost after step 

Even with the smaller stepsize there is quite a lot of noise in measuring the 01 and 10 probabilities. 

In [230]:
params10new = params

dev.shots = 10**6
np.random.seed(0)
circuit(params10new)

array([4.44000e-04, 5.12312e-01, 4.86789e-01, 4.55000e-04])

In [232]:
circuit2(params10new)

array([4.54318724e-04, 5.13142168e-01, 4.85973249e-01, 4.30264281e-04])

In [233]:
dev2.state

array([ 0.02131475+0.j,  0.71633942+0.j, -0.69711782+0.j, -0.02074281+0.j])

In [234]:
cost(params10)

0.0004958784579999998

In [235]:
cost(params10new)

0.0003902014940000004

A marginally smaller cost for 30 times more iterations! To further reduce noise we would need to decrease stepsize even more resulting in more iterations (or find a robust stepsize update schedule).

Finally now we return to the one sample per iteration instance to see how many steps it will take using a fixed stepsize. We will need a much smaller stepsize as mentioned earlier due to the lower bound of the cost function. We use the same stepsize as at the end of prior attempt with the one sample optimization procedure.

In [215]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.001)

# set the number of samples
dev.shots = 1

# set the number of steps
steps = 80000
steps_1 = 80000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)

  if (i+1) % 4000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    print(circuit2(params))

print("Optimized rotation angles: {}".format(params))

Cost after step  4000:  1.5000000
Current rotation angles: [ 0.507 -1.573]
[0.46751874 0.46958381 0.03151803 0.03137942]
Cost after step  8000:  0.5000000
Current rotation angles: [ 1.448 -2.373]
[0.07888574 0.48235823 0.3770866  0.06166943]
Cost after step 12000:  0.5000000
Current rotation angles: [ 1.5685 -2.6465]
[0.03008776 0.4710604  0.46890194 0.02994989]
Cost after step 16000:  0.5000000
Current rotation angles: [ 1.568  -2.7445]
[0.01950708 0.48189108 0.47920355 0.01939829]
Cost after step 20000:  0.5000000
Current rotation angles: [ 1.553  -2.7425]
[0.01999613 0.48890156 0.47180541 0.01929689]
Cost after step 24000:  0.5000000
Current rotation angles: [ 1.573 -2.776]
[0.01648558 0.48241259 0.48454344 0.0165584 ]
Cost after step 28000:  0.5000000
Current rotation angles: [ 1.577 -2.839]
[0.01128776 0.48561042 0.49167313 0.01142869]
Cost after step 32000:  0.5000000
Current rotation angles: [ 1.576  -2.8945]
[0.00755358 0.48984459 0.49496922 0.00763261]
Cost after step 36000:  

In [217]:
params1new = params

dev.shots = 10**6
np.random.seed(0)
circuit(params1new)

array([0.002858, 0.494789, 0.49943 , 0.002923])

In [219]:
circuit2(params1new)

array([0.00287671, 0.49552146, 0.49870664, 0.0028952 ])

In [220]:
dev2.state

array([ 0.05363494+0.j, -0.70393285+0.j, -0.70619164+0.j,  0.05380704+0.j])

In [221]:
cost(params1)

3.275641800000037e-05

In [222]:
cost(params1new)

3.664441400000012e-05

While a valuable learning experience it's unclear if this is an improvement over our prior attempt with updates to the stepsize (with a slightly larger cost!) It takes over 30000 iterations for the 00 and 11 states to fall below 0.01 probability in the theoretical circuit with the learned parameters and with more iterations we should except both to continue to converge to zero. However the stepsize makes the 01 and 10 probabilities noisy and so many more iterations and an even smaller step size would be required (for more stable 01 and 10 probabilities and to drive 00 and 11 probabilities to zero).

Ultimately in this case it appears a variable decreasing stepsize is preferable.

Now summarize succintly the information when using a single stepsize through optimization for each of the various samples.

In [262]:
dev.shots = 10**6
np.random.seed(0)

stepsize_1 = 0.001
steps_1 = 80000
print(cost(params1new))

stepsize_10 = 0.01
steps_10 = 60000
print(cost(params10new))

stepsize_100 = 0.015
steps_100 = 50000
print(cost(params100))

stepsize_1000 = 0.02
steps_1000 = 40000
print(cost(params1000))

4.419151400000023e-05
0.0003428115199999988
7.846465999999965e-06
9.2729400000006e-07


The pattern emerges that as our samples per iteration increase, we can increase the stepsize and lower the iterations with better results. 

The exception is due to the noise in measurements in the 10 samples per iteration instance preventing an improvement from the one sample per iteration case. We make one more attempt with stepsize 0.005 and 70,000 steps for 10 samples per iteration to see if there is an improvement.

In [250]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(0.005)

# set the number of samples
dev.shots = 10

# set the number of steps
steps = 70000

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)

  if (i+1) % 3500 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    print(circuit2(params))

print("Optimized rotation angles: {}".format(params))

Cost after step  3500:  0.0600000
Current rotation angles: [-1.5556  -2.67985]
[0.02657846 0.48101941 0.46661934 0.02578279]
Cost after step  7000:  0.0200000
Current rotation angles: [-1.57125 -2.8677 ]
[0.00931445 0.49045872 0.49090393 0.0093229 ]
Cost after step 10500:  0.0200000
Current rotation angles: [-1.55285 -2.9168 ]
[0.00640279 0.50256989 0.48485028 0.00617704]
Cost after step 14000:  0.0800000
Current rotation angles: [-1.62525 -2.95895]
[0.0039319  0.46885472 0.52282885 0.00438453]
Cost after step 17500:  0.0200000
Current rotation angles: [-1.5894 -3.0062]
[0.00224534 0.48845336 0.50697084 0.00233046]
Cost after step 21000:  0.0000000
Current rotation angles: [-1.5715  -3.00155]
[0.00244577 0.4972024  0.49790263 0.00244921]
Cost after step 24500:  0.0200000
Current rotation angles: [-1.55055 -3.01495]
[0.00204265 0.50807982 0.48791594 0.00196158]
Cost after step 28000:  0.0200000
Current rotation angles: [-1.5665 -3.0368]
[0.00137732 0.50077083 0.4964863  0.00136554]
Cost

In [254]:
params10newest = params
dev.shots = 10**6
np.random.seed(0)
cost(params10newest)

9.933158000000137e-06

Fitting snuggly between our values for the cost for 1 and 100 samples as expected!